<center><h1> Dynamic Contextualization for Starbucks Orders </h1></center>
<center><h2> University of Washington </h2></center>
<center><h3> MS in Data Science</h3></center>
<center><h4> Capstone Project 2022 </h4></center>
<p></p>
<center> Leena Elamrawy | Christie L Gan | Corina Geier | Anant Rajeev | Emily Yamauchi </center> 

## Problem statement

Starbucks deployed a new product recommendation system on their drive-thru screens at 4,000 Starbucks locations across the United States. The goal of this project is to redesign the recommendation system to include dynamic and contextual headlines to increase the conversation rate, thereby increasing sales and ultimately producing higher incremental tickets.

## Our solution

Our system predicts context labels to a requesting store based on conditions like the weather and the time of day. These context labels are then used to filter through potential headlines that would match these recommended products.

Out of all the potential headlines, we picked the one that would give us the highest conversion rate. Our approach involved the use of reinforcement learning in the form of multi-armed bandits to determine the optimal headline choice.   

In [1]:
# load libraries

import pyarrow.parquet as pq
import os

import pandas as pd
import numpy as np

import re

import matplotlib.pyplot as plt
%matplotlib inline

from functions import utils, preprocessing, headlines
from mab import multi_arm_bandits

## Load data

We will be using the sample data from Nov. 11 2021 for this walkthrough.  

As our smaller scope project takes 4 products given as inputs, the data we will be using is mostly static with the exception of weather.

In [2]:
# root path
dirpath = './data/sample_202111111'

# path of parquet files
product_df = utils.get_pq_df(dirpath + '/product.parquet/')
store_df = utils.get_pq_df(dirpath + '/store.parquet/')
ar_df = utils.get_pq_df(dirpath + '/action_reward.parquet/')
weather_df = utils.get_pq_df(dirpath + '/weather.parquet/')

## Preprocessing

The individual datasets can be merged on shared keys, and additional features can be extracted from product names.  

Two important contexts we need to map are the `NotionalFlavor` and `form_codes`. Both are populated sparsely in the original dataset, so for items that do not have labels, we will attempt to derive them from the product names.  

The third preprocessing step is to get the city and state names from the zipcodes given- we are using [this .csv file hosted on GitHub](https://raw.githubusercontent.com/scpike/us-state-county-zip/master/geo-data.csv).

In [3]:
#preprocessing- product df
# get form codes from product names
# if product names contain these keywords, map the form_codes as `Iced`, otherwise, `Hot`
ice_keywords = ['refreshers', 'frappuccino', 'blended', 'cold', 'iced', 'bottled']
product_df2 = preprocessing.get_form_codes(product_df, ice_keywords)

# get notional flavor from product names
prodcut_df2= preprocessing.get_notional_flavor(product_df2)

# preprocessing- store df
# get city state from zip codes
store_df2 = preprocessing.get_zipcodes_from_csv(store_df)

## Inputs given

To generate the dynamic headlines, we are given the following inputs:   

- Recommended products: 4 items
- Store number
- Time of day

In [4]:
# example- given recommended products

products_given = ['chai-tea', 
                  'toasted-white-hot-chocolate', 
                  'clover-x-costa-rica-naranjo-hot',
                  'pumpkin-spice-latte']

In [5]:
# example 2- given recommended products, restricted to iced drinks 

iced_products = ['apple-crisp-frappuccino-blended-beverage',
                 'iced-reserve-bar-caffè-americano',
                 'iced-espresso-classics---skinny-vanilla-latte',
                'toasted-white-chocolate-mocha-frappuccino']

In [48]:
# inputs- time of day (in hours)

hour1 = 8
hour2 = 15
hour3 = 18

# inputs- store numbers
# it was mostly sunny on our sample day in Seattle, hence could not test stores nearby with hot items

store1 = 13507 # drivethru store in longview, WA
store2 = 9447 # this is the one on 15th around interbay/magnolia bridge
store3 = 53313 # the big reserve on michigan ave in chicago

## Generating headlines

The string templates for the headlines are as follows:   

```
{weather_state} in {store_city}
{daypart} {preferred_customer_mode}
{weather_state} {preferred_customer_mode}
{daypart} in {store_city}
{weather_state} {daypart} in {store_city}
```

With 4 possible `preferred_customer_mode`, there are 11 total possible headlines from the template above.  

Our `HeadlineGenerator` class will generate a list of available headlines based on the restrictions posed by the inputs.  

For example, the mode "Treat" will not be applied to products that do not meet the "Treat" profile, or hot drinks will not be recommended on a sunny day.

In [7]:
# initiating HeadlineGenerator: takes the store, product, and weather dataset as inputs

hg = headlines.HeadlineGenerator(store_df2, product_df2, weather_df)

In [49]:
# generate headlines given the store number, time of day, and the list of recommended items
# try with store1

wa_headlines = hg.get_headlines(store1, hour1, products_given)
wa_headlines

['Rainy in Longview',
 'Morning in Longview',
 'Rainy Morning in Longview',
 'Morning Light Pick Me Up',
 'Rainy Light Pick Me Up',
 'Morning Treat',
 'Rainy Treat',
 'Morning Boost',
 'Rainy Boost',
 'Morning Flavor',
 'Rainy Flavor']

The `HeadlineGenerator` will also check to ensure that the products meet the `daypart` and `weather_state` restrictions- i.e. will not recommend a hot drink on a sunny day

In [51]:
# this will result in an error
hg.get_headlines(store2, hour1, products_given)

AssertionError: Drink type does not match weather recommendation

In [53]:
# this will pass
interbay_headlines = hg.get_headlines(store2, hour1, iced_products)
interbay_headlines

['Sunny in Seattle',
 'Morning in Seattle',
 'Sunny Morning in Seattle',
 'Morning Light Pick Me Up',
 'Sunny Light Pick Me Up',
 'Morning Treat',
 'Sunny Treat',
 'Morning Boost',
 'Sunny Boost',
 'Morning Flavor',
 'Sunny Flavor']

In [54]:
# also check caffiene threshold for time of day
# this will result in an error
hg.get_headlines(store3, hour3, iced_products)

AssertionError: Exceeds recommended caffiene threshold

In [56]:
# this will pass
chicago_headlines = hg.get_headlines(store3, hour1, iced_products)
chicago_headlines

['Sunny in Chicago',
 'Morning in Chicago',
 'Sunny Morning in Chicago',
 'Morning Light Pick Me Up',
 'Sunny Light Pick Me Up',
 'Morning Treat',
 'Sunny Treat',
 'Morning Boost',
 'Sunny Boost',
 'Morning Flavor',
 'Sunny Flavor']

## Simulation: runing the Multi-Armed Bandits model

The multi-armed bandit problem models an agent that wants to simultaneously acquire new knowledge (exploration) and optimize decisions based on existing knowledge (exploitation).

The information (customers purchasing the recommended products or not) gained from the data gathered will shift the distribution in different ways to reflect an accurate reward model.

The model recursively continues to learn about which headlines are contributing to a higher conversion rate. This then allows Starbucks to make a store-by-store decision with regards to which headlines they want to display on that drive-thru.   

We were faced with the problem of a cold start (unsupervised learning) due to the lack of historical data regarding screen engagement with these newly generated headlines. To combat that, we adopted the Thompson Sampling algorithm, which uses a beta distribution as a parametric assumption to model the prior unknown distribution. The model recursively continues to learn and adjusts the reward model as a result.

In [59]:
# parameters: headlines, num_turns, lbound=0.01, ubound=0.15
# initiate the mab model

mab = multi_arm_bandits.MultiArmBandits(wa_headlines, num_turns=1000)

In [60]:
# run the simulation
# returns the conversions and fails

mab_conversion, mab_fails = mab.simulation(verbose=True)

Headline 0 was chosen 40.0 times
Headline 1 was chosen 27.0 times
Headline 2 was chosen 29.0 times
Headline 3 was chosen 41.0 times
Headline 4 was chosen 60.0 times
Headline 5 was chosen 40.0 times
Headline 6 was chosen 521.0 times
Headline 7 was chosen 82.0 times
Headline 8 was chosen 27.0 times
Headline 9 was chosen 74.0 times
Headline 10 was chosen 59.0 times

Overall conclusion: best headline is 6

Rainy Treat


In [64]:
mab_interbay = multi_arm_bandits.MultiArmBandits(interbay_headlines, num_turns=1000)

In [65]:
mab_interbay_conv, mab_interbay_fails = mab_interbay.simulation(verbose=True)

Headline 0 was chosen 133.0 times
Headline 1 was chosen 141.0 times
Headline 2 was chosen 37.0 times
Headline 3 was chosen 107.0 times
Headline 4 was chosen 28.0 times
Headline 5 was chosen 148.0 times
Headline 6 was chosen 66.0 times
Headline 7 was chosen 65.0 times
Headline 8 was chosen 128.0 times
Headline 9 was chosen 41.0 times
Headline 10 was chosen 106.0 times

Overall conclusion: best headline is 5

Morning Treat


## Next steps

We are currently setting the parameters using an arbitrary threshold for many of the methods- such as the caffiene, sugar, calorie thresholds, or the distribution of the simulated headlines. We could take a better approach to fine-turning these parameters.   

Our methodology to categorize the iced and hot drinks can also be improved.  

Another major next step would be to generate the recommended drinks for each store based on the `action_rewards` dataset- due to project timeline constraints, we accepted the products as given inputs, but had we more time we would like to find and learn the best products.